Marcin Wardyński
czwartek, 8:00

## Laboratorium 10 - RAG

Testy jakości odpowiedzi oraz wydajności RAGa wykonam w oparciu o dwa modele: `Mistral` oraz `Llama 3.2`

W poniższym kodzie inicjalizuję embeddingi dla Ollama z Langchain:

In [360]:
from langchain_ollama import OllamaEmbeddings

model_name = "mistral" # llama3.2

embeddings = OllamaEmbeddings(model=model_name)


Poniżej tworzę wektorową bazę danych `Qdrant` dla gęstych wektorów osadzenia, oraz definiuję kolekcję: `aspirin`:

In [361]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

vecotor_size =  4096 # 3072
collection_name = "aspirin-m-1500"


client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vecotor_size, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE
)

Testy przeprowadzę dla dwóch różnych rozmiarów chunków: 500 i 1500.

Poniższa funkcja dzieli tekst z podanego pliku pdf na fragmenty o długości możliwie bliskich zadanym:

In [362]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1500 # 1500

def load_chunks(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size/5,
        length_function=len
    )

    return text_splitter.split_documents(documents)

W tym bloku tekst zostaje przetworzony na osadzenia wektorowe i zapisany w stworzeonej uprzednio kolekcji. Dodatkowo mierzę czas indeksowania pliku.  

Plikiem użytym w ćwiczeniu będzie ulotka informacyjna do leku Aspiryna C. Wybrałem taki plik, gdyż zawiera on szerag informacji, na których można testować działanie RAGu, a dodatkowo działanie aspiryny jest szeroko znane, a wiedzę o samym leku można uznać za wiedzę ogólną. Fakt o wiedzy ogólnej chcę wykorzystać w porównaniu RAGa z czystym modelem językowym dla swoistego wyrównania szans, a przynajmniej dla zniwelownia znacznej przewagi jaką uzyskuje RAG otrzymując kontekst dla części generwatywnej. Używając wyłącznie modelu językowego w mojej konfiguracji, cała ulotka jest zbyt długa, by zawrzeć ją w kontekście, więc przeprowadzę test "closed-book" i będę bazował wyłącznie na wewnętrznej wiedzy modelu mając nadzieję, że w trakcie treningu poznał on kilka informacji o aspirynie.

In [363]:
import time

def vectorize_pdf_content(pdf_path):
    chunks = load_chunks(pdf_path)
    _ = vector_store.add_documents(documents=chunks)

start_time = time.time()

pdf_path = "Aspirin C.pdf"
vectorize_pdf_content(pdf_path)

execution_time = time.time() - start_time

print(f"execution time: {execution_time}")

execution time: 40.12127685546875


Poniższa tabela przedstawia czas indeksowania w zależności od rozmiaru chunka i użytego modelu dla osadzeń wektorowych:

| rozmiar chunka | czas indeksowania - mistral | czas indeksowania - llama3.2 |
|----------------|-----------------------------|------------------------------|
| 500            | 37.7s                       | 17.5s                        |
| 1500           | 40.1s                       | 16.5s                        |

Jak widzimy, model `Mistral` o 7 miliardach parametrów ma ponad dwókrotnie dłuższy czas indeksowania, niż `Llama 3.2` 3 miliardach parametrów. Na tym przykłazie możnaby stwierdzić, że czas indeksowania rośnie proporcjonalnie do wzrostu liczby parametrów modelu.  
Wielkość chunków ma niewielki wpływ na czas trwania indeksowania.

Poniższy blok implementuje element Retrievera, który zwaraca trzy fragmenty tekstu z indeksu.

In [364]:
from langchain_core.runnables import RunnablePassthrough

retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_runnable = {"question": RunnablePassthrough(), "context": retriever}

Zadanie wykonam na kilku promptach dla modelu językowego. Oto rodzaje promptów, które chcę wypróbować:
- Zero-shot jedynie z pytaniem i krótkim poleceniem dla modelu
- Zero-shot z dodatkową specyfikacją zachowania modelu
- Few-shot z jednym przykładem i specyfikacją zachowania modelu

In [404]:
template_1_rag = """
Jesteś asystentem w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.

Kontekst: {context}
Pytanie: {question}
Odpowiedź:
"""

template_2_rag = """
Jesteś asystentem w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.
Udzielaj zwięzłych odpowiedzi i upewnij się, że twoja odpowiedź jest mocno powiązana z dostarczonym kontekstem.

Kontekst: {context}
Pytanie: {question}
Odpowiedź:
"""

template_3_rag = """
Jesteś asystentem w zadaniu odpowiadania na pytania. Użyj podpowiedzi zawartej w kontekście.
Udzielaj zwięzłych odpowiedzi i upewnij się, że twoja odpowiedź jest mocno powiązana z dostarczonym kontekstem.
Wzoruj się na poniższych przykładach:

Przykład 1:
Kontekst: Prowadzenie pojazdów i obsługiwanie maszyn: Lek nie ma wpływu.
Pytanie: Czy po zażyciu Aspirin C można prowadzić samochód?
Odpowiedź: Tak, lek nie ma wpływu na możliwość prowadzenia samochodu.

Przykład 2:
Kontekst: Jedna tabletka musująca zawiera 467 mg sodu (głównego składnika soli kuchennej).
Pytanie: Ile mg sodu zawiera jedna tabletka?
Odpowiedź: Jedna tabletka zawiera 467 mg sodu.


Przykład 3:
Kontekst: W przypadku pominięcia dawki leku i utrzymywania się dolegliwości należy przyjąć następną dawkę leku Aspirin C. Nie należy stosować dawki podwójnej w celu uzupełnienia pominiętej dawki.
Pytanie: Co zrobić, gdy dawka leku została pominięta?
Odpowiedź: Należy przyjąć następną dawkę i nie stosować dawki podwójnej.

Twoje zadanie:
Kontekst: {context}
Pytanie: {question}

Odpowiedź:
"""

In [396]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage
from langchain_ollama.llms import OllamaLLM

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template_1_rag
)

model = OllamaLLM(model=model_name)

generator_runnable = prompt | model | StrOutputParser()


In [397]:
rag_chain = retriever_runnable | generator_runnable

Pytania do sprawdzenia działania RAGa:
- Jaką substancję czynną zawiera Aspirin C?
- W jakim wieku można przyjmować Aspirin C?
- Czy kobiety ciężarne mogą przyjmować ten lek?
- Czy lek można łączyć z alkoholem?
- Jak przechowywać Aspirin C?

Odpowiedzi:
- kwas acetylosalicylowy oraz witaminę C
- od 12 roku życia
- W trzecim trymestrze mogą, a w pierwszym i drugim nie powinny
- Alkohol może zwiększyć ryzyko wystąpienia działań niepożądanych ze strony przewodu pokarmowego, tj. owrzodzenie błony śluzowej lub krwawienia.
- Przechowywać w temperaturze poniżej 25ºC.

In [403]:
question = "Jak przechowywać Aspirin C?"

result = rag_chain.invoke(question)
print(result)

 Aspirin C należy przechowywać w temperaturze pomiędzy 15 °C a 25 °C. Nie należy przechowywać leku w miejscu ciemnym lub w miejscu, gdzie może zostać dotknięty promieniowaniem UV. Zalecana jest przetrzymywanie opakowania w oryginalnej pudełce i zachowanie przy sobie recepty leku.


Model: `Mistral`, rozmiar chunk-ów: 1500 znaków, numer prompta: 1, czas wykonania: 12s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Kwas acetylosalicylowy (aspiryna) jest substancją czynną zawartą w leku Aspirin C.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Można przyjmować lek Aspirin C od najmłodszych dzieci poza karmieniem piersią do osób starszych. Jednak należy zachować szczególną ostrożność i konsultować się z lekarzem, gdy dziecko jest młodsze niż 12 lat lub ma jakiekolwiek inne schorzenia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: * Można przypuścić, że w kontekście zaoferowanym nie podano informacji o stosowaniu leku Aspirin C przez kobiety ciężarne. W celu ustalenia odpowiedzi należy wykonać dalsze poszukiwanie informacji w kontekście, który jest dostępny lub skontaktować się z lekarzem.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: * Tak, jednak z powagą należy unikać spożywania alkoholu podczas stosowania Aspirinu C oraz innych niesteroidowych leków przeciwzapalnych w dużych dawkach, ponieważ może to wzmagać ryzyko krwawień z przewodu pokarmowego.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: * Aspirin C powinien być przechowywany w suchym i ciemnym miejscu, przy odpowiednio niskiej temperaturze. Należy zachować szczególną ostrożność w przypadku składowania leków w miejscu dostępnym dla dzieci lub zwierząt, ponieważ może to prowadzić do nieszczędnego sporzenia.*

===================================================================

Model: `Mistral`, rozmiar chunk-ów: 1500 znaków, numer prompta: 2, czas wykonania: 13s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Kwas acetylosalicylowy jest substancją czynną w leku Aspirin C.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Można przyjmować Aspirin C w dowolnym wieku, jednak należy zachować szczególną ostrożność u dzieci.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie jest zalecane stosowanie leku Aspirin C u kobiet w ciąży, ponieważ może zwiększać ryzyko wystąpienia krwotoków i złej sztuczki. Zawsze należy skonsultować się z lekarzem dotycząco stosowania leków w ciąży.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, kwas acetylosalicylowy (łac. Aspirin), osłabia działanie alkoholu i powoduje zwiększone ryzyko wystąpienia dny moczanowej, jeśli jest stosowany jednocześnie z nim.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinien być przechowywany w miejscu suchym, przytłoczonym i umiarkowanych temperaturach. Zaleca się trzymanie go poza zasięgiem światła słonecznego oraz daleko od innych leków i produktów chemicznych.*

===================================================================

Model: `Mistral`, rozmiar chunk-ów: 1500 znaków, numer prompta: 3, czas wykonania: 14s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera jako substancję czynną kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Można przyjmować lek Aspirin C w dowolnym wieku.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: * Nie, kobiety ciężarne nie powinny przyjmować tego leku, ponieważ może on być szkodliwy dla płodu.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Tak, jednak należy zachować szczególną ostrożność w stosowaniu Aspirin C wraz z alkoholem, ponieważ może to prowadzić do zwiększonego ryzyka krwawień.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *  Aspirin C należy przechowywać w temperaturze pomiędzy 15 °C a 25 °C. Nie należy przechowywać leku w miejscu ciemnym lub w miejscu, gdzie może zostać dotknięty promieniowaniem UV. Zalecana jest przetrzymywanie opakowania w oryginalnej pudełce i zachowanie przy sobie recepty leku.*

Model: `Mistral`, rozmiar chunk-ów: 500 znaków, numer prompta: 1, czas wykonania: 7s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Acidum acetylsalicylicum oraz Acidum ascorbicum (witamina C)*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: * Można zacząć przyjmować lek Aspirin C od lat 12 i więcej, jednak należy zachować szczególną ostrożność i przed zawarciem umowy o leczeniu z lekarzem lub farmaceutą przeczytać ulotkę zawartą w opakowaniu.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: * Tak, według treści dokumentu "Aspirin C", u kobiet w ostatnim trymestrze ciąży należy unikać stosowania tego leku. Jednak jeśli podano inną dawkę leku metotreksat (np. 15 mg na tydzień lub większą), to kobieta może być zastraszona z jego użycia. Zawsze należy sięgnieć do zaleceń lekarza lub farmaceuty.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Z konttekstu nie zostało żadnie wskazówki dotyczące łączenia aspiryny C z alkoholem. Moja odpowiedź jest tylko hipotetyczna, ponieważ konieczne jest zażywanie leków w sposób zgodny z rekomendacjami lekarza lub farmaceuty. Wobec braku informacji na temat interakcji aspiryny C i alkoholu, należy skonsultować się z lekarzem lub farmaceuty.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: * Aby zapewnić poprawne przechowywanie tabletek Aspirin C, należy je przechowywać w temperaturze pomiędzy 15 °C a 25 °C. Miejsce przechowywania powinno być suche i niezawierające światła słonecznego, tak aby tabletki utrzymywały swoją jakość na odpowiednim poziomie. W razie potrzeby, zachowanie dokumentacji ze szczegółami dotyczącymi receptury i informacji dla pacjenta jest również istotne.*

===================================================================

Model: `Mistral`, rozmiar chunk-ów: 500 znaków, numer prompta: 2, czas wykonania: 7s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera dwa związki czynne: Acidum acetylsalicylicum (aspirynę) oraz Acidum ascorbicum (witaminę C).*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C nie powinno być stosowane u dzieci poniżej 12 lat (z kontekstu).*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Tak, kobiety w ostatnim trymestrze ciąży powinny unikać stosowania tego produktu (Aspirin C).*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: * Informacja o możliwości stosowania Aspirin C w połączeniu z alkoholem nie została dostarczona w kontekście. Zaznaczono jedynie przeciwwskazania do stosowania leku u osób w ostatnim trymestrze ciąży, u dzieci poniżej 12 lat oraz osób stosujących metotreksat w dawkach ponad 15 mg na tydzień. W razie wątpliwości związanych ze stosowaniem leku należy zwrócić się do lekarza lub farmaceuty.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: * Aspirin C powinny być przechowywane w suchym i zimnym miejscu, poza bezpośrednim nasłonecznieniem. W razie potrzeby należy upewnić się, że dostęp do leku jest ograniczony, aby uniknąć nieprzyjemnych skutków ze spożycia przypadkowego przez inne osoby.*

===================================================================

Model: `Mistral`, rozmiar chunk-ów: 500 znaków, numer prompta: 3, czas wykonania: 7s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: * Aspirin C zawiera Acidum acetylsalicylicum (acetylsalicylan sodu) oraz Acidum ascorbicum (kwas witaminowy C).*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Można przyjmować Aspirin C od lat 12 i więcej.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, kobiety w ostatnim trymestrze ciąży nie powinny stosować tej preparaty Aspirin C.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie należy łączyć leku Aspirin C z alkoholem, ponieważ to może skutkować powikłaniami.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Należy przechowywać Aspirin C dokładnie jak opisano w ulotce dołączonej do opakowania leku, ponieważ zawiera ona informacje ważne dla pacjenta.*

Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 1, czas wykonania: 6s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną w leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C nie może być przyjmowany u dzieci poniżej 12 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Kobiety ciężarne nie powinny przyjmować leku Aspirin C, poza przypadkami w których konieczna jest jego przyjęcie za zgodą lekarza.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C nie powinien być łączony z alkoholem. Alkohol może pogorszyć efekty leku i powodować negatywne interakcje.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Niestety, nie ma informacji o sposobach przechowywania leku Aspirin C w dostępnym treści.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 2, czas wykonania: 6s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera jako substancję czynną kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C powinien być przyjmowany tylko przez osoby w wieku ponad 12 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny przyjmować tego leku.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C powinien być stosowany bezpośrednio przed lub po przyjęciu alkoholu. Alkohol może rozwijać niepożądane działania leku, takie jak zwiększone ryzyko niesawietności i śpiączki.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Przestrzeń magazynowej drygą i szubienicą na wysokości 15-20°C i w miejscach nie narażonych dla promieni UV.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 1500 znaków, numer prompta: 3, czas wykonania: 7s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Lek Aspirin C zawiera kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *U dzieci poniżej 12 roku życia leków zawierających kwas acetylosalicylowy nie należy stosować, a u młodzieży powyżej 12 roku życia leki zawierające kwas acetylosalicylowy można stosować wyłącznie na zlecenie lekarza.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie, lek Aspirin C nie powinien być wykorzystywany przez kobiety w trakcie ciąży i w okresie późniejszym po porodzie.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie, lek Aspirin C nie może być łączony z alkoholem. W razie jakichkolwiek wątpliwości związanych ze stosowaniem tego leku należy zwrócić się do lekarza lub farmaceuty.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Lek na receptę powinien być przechowywany w miejscu niedostępnym dzieciom i osobom niepełnoletnimi.*

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 1, czas wykonania: ok 2.5s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Lek Aspirin C może być przyjmowany przez dorośli w wieku od 1 roku życia.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie należy stosować leków zawierających kwas acetylosalicylowy u kobiet w pierwszym i drugim trymestrze ciąży, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Nie ma informacji w dokumentach o sposobie przechowywania leku Aspirin C.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 2, czas wykonania: ok 2.5s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera kwas acetylosalicylowy jako substancję czynną.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C powinno być przepisane tylko dla dorosłych osób zgodnie z opisem w ulotce, a nie dla dzieci. Oznacza to, że Aspirin C nie powinno być przyjmowane przez dzieci poniżej 12 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie. Kobiety w pierwszym i drugim trymestrze ciąży nie powinny zawsze przyjmować leku Aspirin C, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie jest zalecane łączyć leku Aspirin C z alkoholem.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Odpowiedź nie jest dostępna w kontekście podanym. Kontekst opisuje korzystanie z leku Aspirin C i jego dawkowanie, ale nie omawia sposobu przechowywania leku.*

===================================================================

Model: `Llama 3.2`, rozmiar chunk-ów: 500 znaków, numer prompta: 3, czas wykonania: ok 4s
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną leku Aspirin C jest kwas acetylosalicylowy.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *W związku z treścią dokumentu, aspektem, który powinien być uwzględniony, kiedy dziecko może przyjąć Aspirin C to wiek 12 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować ten lek?*  
Odpowiedź: *Nie. Według ulotki do pacjenta, nie należy stosować leków zawierających kwas acetylosalicylowy u kobiet w pierwszym i drugim trymestrze ciąży, chyba że jest to bezwzględnie konieczne.*

Pytanie: *Czy lek można łączyć z alkoholem?*  
Odpowiedź: *Nie. Lek Aspirin C nie powinien być łączony z alkoholem, ponieważ może przyczynić się do zwiększenia ryzyka zatrucia.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Z powodu braku informacji na temat przechowywania leku Aspirin C w kontekście zadanejapytania, nie udzielę konkretnych zaleceń.*

Poniżej definiuję szablony do użycia z modelem językowym, który występuje samodzielnie i nie jest częścią RAGa. Szablony odpowiadają tym dla RAGa, tylko została z nich usunięta sekcja odpowiedzialna za kontekst.  
Dodatkowo do intearkcji z czystym językiem modelowy, zmodyfikowałem trochę pytania, tak żeby w każdym znajdowało się jasne wskazanie nazwy leku: *Aspirin C*.

In [389]:
template_1_lm = """
Jesteś asystentem w zadaniu odpowiadania na pytania.

Pytanie: {question}
Odpowiedź:
"""

template_2_lm = """
Jesteś asystentem w zadaniu odpowiadania na pytania. Udzielaj zwięzłych odpowiedzi.

Pytanie: {question}
Odpowiedź:
"""

template_3_lm = """
Jesteś asystentem w zadaniu odpowiadania na pytania. Udzielaj zwięzłych odpowiedzi.
Wzoruj się na poniższych przykładach:

Przykład 1:
Pytanie: Czy po zażyciu Aspirin C można prowadzić samochód?
Odpowiedź: Tak, lek nie ma wpływu na możliwość prowadzenia samochodu.

Przykład 2:
Pytanie: Ile mg sodu zawiera jedna tabletka?
Odpowiedź: Jedna tabletka zawiera 467 mg sodu.

Przykład 3:
Pytanie: Co zrobić, gdy dawka leku została pominięta?
Odpowiedź: Należy przyjąć następną dawkę i nie stosować dawki podwójnej. 

Twoje zadanie:
Pytanie: {question}
Odpowiedź:
"""

In [423]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

model_name="llama3.2"
question = "Czy Aspirin C można łączyć z alkoholem?"

prompt = ChatPromptTemplate.from_template(template_3_lm)

model = OllamaLLM(model=model_name)

lm_chain = prompt | model | StrOutputParser()

lm_chain.invoke({"question": question})

'Nie, Aspirin C powinna być spożywana bez alkoholu. Alkohol może pogorszyć efekt leku i popromienieje ryzyko wystąpienia powikłań.'

Model: `Mistral`, numer prompta: 1, czas wykonania: 4s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspiryna (acetylosalicylowa kwasu) jest substancją czynną w Aspirin C. Jest to lek należący do grupy salicylanów, stosowany przede wszystkim jako środek przeciwbólowy i przeciwzapalny.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C, czyli Akcetylosalicylan potasu (Aspiryn Potasu) jest lekiem z receptury i nie powinno się dać dzieciom poniżej 12 roku życia bez zgody lekarza. Dorośli mogą przyjmować Aspiryn C po 325 mg raz na dobę, ale należy wspierać to przepisem lekarskim. Nie zapomnij o odpowiednim posiłku i dbałości o zachowanie stosowania leku pod nadzorem lekarza.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *Nie, kobiety ciężarne powinny unikać spożywania Aspirynu C z powodu ryzyka krwawienia u macicy oraz ryzyka uszkodzenia płodu. Jeżeli kobieta cierpi na zatorowość nerek, powinniśmy się ze sobą skontaktować i porozmawiać o możliwościach leczenia alternatywnymi środkami.*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Nie należy łączyć aspiryny z alkoholem, gdyż może to spowodować zwiększenie ryzyka uszkodzeń żołądka i płuc. Zapotrzebowanie na leki zmniejsza się przy spożywaniu alkoholu, więc nie jest to najlepsze rozwiązanie, jeśli potrzebujesz lekarstwa. Jeśli musisz zadbać o swoją zdrowie i brać aspirynę, to należy dbać o gruntowną dietyę i odwiedzać lekarza.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirynę należy przechowywać w suchym miejscu, przy temperaturze pomiędzy 15-25 stopni Celsjusza (59-77 stopni Fahrenheita). Nie należy przetrzymywać jej w bezpośrednim sąsiedztwie przesłanek, które mogą uwalniać wilgoć. W celu zachowania leczniczej działności Aspiryny powinna być chroniona przed światłem słonecznym i od warunków wysokiej wilgotności.*

=========================================================

Model: `Mistral`, numer prompta: 2, czas wykonania: 4s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspiryna C zawiera aketylsalicylowę (Acetylsalicylowe) jako substancję czynną.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspiryny dawkowane powinny być brane pod opieką lekarza i należy uważać na wiek, waga oraz zdrowie ogólne pacjenta. Nie zalecam samodzielnego rozstrzygania takiego kwestii.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny brać aspiryny C bez wcześniejszego konsultowania się z lekarzem. Może to spowodować powikłania dla samych matki i płodu.*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Nie należy mieszanać Aspiryn C i alkohol, gdyż to może prowadzić do poważnych skutków zdrowotnych.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirynę powinnośmy przechowywać w suchym miejscu, przy stałej temperaturze pomiędzy 20°C a 30°C. Należy unikać umieszczania jej w miejscach nasłonecznionych lub przysłoniętych. Zalecane jest zachowywanie jej w oryginalnej oprawce, aby zapobiec powstawaniu prążków i utracie skuteczności.*

=========================================================

Model: `Mistral`, numer prompta: 3, czas wykonania: 4s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera aketonasalicylanowę (salicylanylitową).*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Można przyjmować Aspirin C od najmłodszych dzieci. Podczas gdy dorosłe osoby mogą brac Aspirin C w dowolnym wieku, należy zawsze uważać na podania lekarza.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *Tak, kobiety ciężarne mogą przyjmować Aspirin C pod nadzorem lekarza.*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Tak, Aspirin C może być konsumowany razem z alkoholem, jednak może to spowodować wzrost poziomu krwi alkoholu i osłabienie jego działania przeciwbólowego. Zaleca się unikanie mieszania leków z alkoholem.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Należy przechowywać Aspiryn C w suchym miejscu i przy temperaturze nie przekraczającej 30 stopni Centigrada.*

Model: `Llama 3.2`, numer prompta: 1, czas wykonania: 1s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną w Aspirine C jest acetylozyd kwasu parabenzojsowego (ACETAMINOFEN).*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C (w postaci tabletek lub rozcieńczonych w lekach) jest stosowana głównie u dzieci i młodych dorosłych. Optymalna wieku do przyjęcia Aspirin C to:\n\n- Dzieci w wieku od 6 lat, jeżeli są zdiagnozowane choroba w której Aspirin C jest zaproponowany przez lekarza.\n- Dzieci w wieku od 2-5 lat mogą otrzymywać Aspirin C pod nadzorem lekarskim i dopiero wtedy, jeśli zdiagnozowana choroba to powyższy wiek.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *'Nie rekomenduje się kobietem ciężarnym przyjmowania Aspirinu C, ponieważ może ono przyczynić się do zwiększenia ryzyka powikłań w trakcie ciąży i podczas porodu.\n\nAspirin jest antytrombotycznym lekiem, który może uszkadlać naczynia krwiotoczne i spowodować niepowodzenia w trakcie ciąży. W szczególności, Aspirin C (zawierający acetylosalicylan i kwas cytrynowy) jest dodatkowo ryzykowny dla ciężarnych kobiet.\n\nW przypadku ciąż niezależnie od stanu ciąży lek ten powinien być zawsze skonsultowany z lekarzem.'*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Nie jestem w stanie odpowiedzieć jako lekarz, ale ogólna informacja.\nAlkohol i Aspirin C nie powinny być łączone, ponieważ alkohol może przyspieszyć utratę cieczu z organizmu i podwoić ryzyko nadmiernego zastrzyku aspiryny.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinny być przechowywane w lodzie, aby utrzymać skuteczność leku.*

=========================================================

Model: `Llama 3.2`, numer prompta: 2, czas wykonania: 1s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Aspirin C zawiera fenacetyd w ilości około 80%.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Aspirin C można przyjmować od 6 roku życia, ale powinno być poddawane szczególnym uzgodnieniom z lekarzem.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny przyjmować aspirinu bez specjalnej indications i pod określonych warunkach. Aspirynę może przyczyniać się do hemoragii w czasie porodów lub połogów, co może być niebezpieczne dla matki i niestrukturalnie dla jej dziecka.*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Nie, Aspirin C nie powinno być łączone z alkoholem, ponieważ może wzmacniać efekt alkoholu i powodować negatywne efekty na zdrowie.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinien być przechowywany w suchym i ciemnym miejscu, odlotnie od źródła światła. Powinien być chroniony przed wilgomością i mechanicznym uszkodzeniem.*

=========================================================

Model: `Llama 3.2`, numer prompta: 3, czas wykonania: 2s  
Pytanie: *Jaką substancję czynną zawiera Aspirin C?*  
Odpowiedź: *Substancją czynną Aspirin C jest acetylosalicylan.*

Pytanie: *W jakim wieku można przyjmować Aspirin C?*  
Odpowiedź: *Wiek, w którym można przyjmować Aspirin C, to od 16 lat.*

Pytanie: *Czy kobiety ciężarne mogą przyjmować Aspirin C?*  
Odpowiedź: *Nie, kobiety ciężarne nie powinny przyjmować Aspirin C bez zgody lekarza.*

Pytanie: *Czy Aspirin C można łączyć z alkoholem?*  
Odpowiedź: *Nie, Aspirin C powinna być spożywana bez alkoholu. Alkohol może pogorszyć efekt leku i popromienieje ryzyko wystąpienia powikłań.*

Pytanie: *Jak przechowywać Aspirin C?*  
Odpowiedź: *Aspirin C powinno być przechowywane w miejscu drymowym, na wysokości 10-20°C. Unikaj jego zastygania w temperaturze poniżej 8°C lub przekroczenia 25°C.*